In [ ]:
from tqdm.auto import tqdm
import numpy as np

def process_chunk(df, ds_merge, step):

    # Get coordinates that exist in both df_temp and pv_df
    common_lats = np.intersect1d(ds_merge.lat.values, df['lat'].unique())
    common_lons = np.intersect1d(ds_merge.lon.values, df['lon'].unique())

    # Select data from df_temp using only the common coordinates
    ds_merge = ds_merge.sel(
        lat=common_lats,
        lon=common_lons,
    )

    #step = 500
    total_lat = len(ds_merge.lat)
    total_lon = len(ds_merge.lon)
    merged_dfs = []
    
    # Calculate total iterations for progress bar
    total_iterations = (total_lat // step + (1 if total_lat % step else 0)) * \
                    (total_lon // step + (1 if total_lon % step else 0))

    # Create progress bar with Chinese description
    pbar = tqdm(total=total_iterations, desc="处理数据块")

    # Iterate through all latitude and longitude points in chunks
    for start_lat in range(0, total_lat, step):
        end_lat = min(start_lat + step, total_lat)
        for start_lon in range(0, total_lon, step):
            end_lon = min(start_lon + step, total_lon)
            #print(1)
            # Extract data from ds_merge for each time point
            df = ds_merge.isel(
                lat=slice(start_lat, end_lat),
                lon=slice(start_lon, end_lon)
            ).compute().to_dataframe()
            
            # Reset index to convert multi-index to columns
            df = df.reset_index()
            
            # Merge with pv_df based on lat/lon coordinates
            #chunk_merged = pd.merge(df, pv_df, on=['lat','lon'], how='inner')
            merged_dfs.append(df)
            
            # Update progress bar
            pbar.update(1)
            
    # Close progress bar
    pbar.close()
    # Combine all chunks into final dataframe
    merged_df = pd.concat(merged_dfs, ignore_index=True)
    return merged_df
process_chunk(df, ds_merge,50,500)
